# Run: pvert5 - Vertebrate and Viromes

```
Lead     : ababaian
Issue    : 
Version  : v0.3.5-dev : ab-dev branch
start    : 2020 09 05
complete : 2020 09 ..
files    : ~/serratus/notebook/200905_ab/
s3_files : s3://serratus-public/notebook/200905_ab/
output   : s3://serratus-public/out/200905_pvert5/
```

### Intro/Objectives

- Run protref5 against vertebrate RNA-seq and Virome extended SRA queries


## Virome + Metatranscriptome

Query: `"VIRAL METAGENOME" OR "VIROME" OR "VIROMIC" OR "VIRAL RNA" OR "METATRANSCRIPTOMIC" NOT "METAGENOMIC" NOT amplicon[All Fields] AND "platform illumina"[Properties] AND cluster_public[prop]`

Date: `2000904`
Return: `60327`

## Vertebrata and Mammalia

Query: `("Vertebrata"[Organism] NOT "Homo sapiens"[Organism] NOT "Mus musculus"[orgn]) AND ("type_rnaseq"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]`

Date: `200905`
Return: `93981`


Saved to "$WORK"

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200905_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200905_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sat Sep  5 10:37:36 PDT 2020
fde8446fbf106afff5ec1c82941f154fc39c97f7


In [4]:
# Deplete duplicate entries from previous pmeta5 run
cd $WORK
aws s3 ls s3://serratus-public/out/200830_pmeta5/psummary/ > pmeta5.sra

In [13]:
# Reduce to completed SRA accessions
sed 's/.* //g' pmeta5.sra \
  | sed 's/.psummary//g' - \
  | sed 's/$/,/g' - \
  | cut -f1 -d'.' - > pmeta5.sra.complete

In [14]:
head -n1 vert_SraRunInfo.csv > sra.header

# Inverse select viro and vert
grep -vif pmeta5.sra.complete vert_SraRunInfo.csv  | tail -n+2 - > vert5_SraRunInfo.csv
grep -vif pmeta5.sra.complete viro_SraRunInfo.csv  | tail -n+2 - > viro5_SraRunInfo.csv

cat sra.header vert5* viro5* > v5_SraRunInfo.csv

wc -l *
md5sum *

   499380 pmeta5.sra
   499380 pmeta5.sra.complete
        1 sra.header
   162011 v5_SraRunInfo.csv
   100545 vert5_SraRunInfo.csv
   102342 vert_SraRunInfo.csv
    61465 viro5_SraRunInfo.csv
    61466 viro_SraRunInfo.csv
  1486590 total
78f7ab38f98bf0534d176496c7332ce6  pmeta5.sra
821d3fc25ff8a8274e463b67e4a0df49  pmeta5.sra.complete
0a9601837e2e43ec3cc4472943eea408  sra.header
9b587e50581701710eac061b59b5770a  v5_SraRunInfo.csv
6e36bc31d4f6746d4b5905001c5f6066  vert5_SraRunInfo.csv
628efc901934c56be01954d08aef059f  vert_SraRunInfo.csv
5873be9feb9aee096ad2e56bc26a6c8b  viro5_SraRunInfo.csv
2ddb64e9f5706fd64354a45f296592b4  viro_SraRunInfo.csv


In [15]:
rm sra.header vert5_SraRunInfo.csv viro5_SraRunInfo.csv

md5sum * > v5.md5sum

In [16]:
aws s3 sync ./ $S3_WORK

upload: ./v5.md5sum to s3://serratus-public/notebook/200905_ab/v5.md5sum
upload: ./pmeta5.sra.complete to s3://serratus-public/notebook/200905_ab/pmeta5.sra.complete
upload: ./pmeta5.sra to s3://serratus-public/notebook/200905_ab/pmeta5.sra
upload: ./viro_SraRunInfo.csv to s3://serratus-public/notebook/200905_ab/viro_SraRunInfo.csv
upload: ./vert_SraRunInfo.csv to s3://serratus-public/notebook/200905_ab/vert_SraRunInfo.csv
upload: ./v5_SraRunInfo.csv to s3://serratus-public/notebook/200905_ab/v5_SraRunInfo.csv


### Terraform Initialize

In [18]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..c834ce6 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@ re

In [19]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [20]:
cd $WORK
BATCH='v5_SraRunInfo.csv'
wc -l $WORK/$BATCH

162011 /home/artem/serratus/notebook/200905_ab/v5_SraRunInfo.csv


In [22]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200905_ab/v5_SraRunInfo.csv
  date: Sat Sep  5 11:16:16 PDT 2020
  wc  : 162011 /home/artem/serratus/notebook/200905_ab/v5_SraRunInfo.csv
  md5 : 9b587e50581701710eac061b59b5770a  /home/artem/serratus/notebook/200905_ab/v5_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
7d68861f9c2a38cdcb178ab8a4e522ce  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
e48e056d67e858d4ae836cb4d314682d  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
e5efc0b23f41cbbed061c91cce4ac560  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
b0ad2afb614781d918bca8a768df47bd  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"to

## Run Serratus

In [24]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.15,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3000,
  "CLEAR_INTERVAL": 600,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 600,
  "GENOME": "protref5",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 50,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 200,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 45
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":25,"ALIGN_SCALING_CONSTANT":0.15,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":3000,"CLEAR_INTERVAL":600,"DL_ARGS":"","DL_MAX

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
cd $TF

aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [ ]:
# Exploring sumbler data
# DO NOT UNCOMMENT WILL DELETE MANY FILES
# aws s3 sync s3://serratus-public/out/200823_test/sumbler/ ./
# find . -size -2k -exec rm '{}' \;
# grep 'SRAsum' * | sed 's/;/\t  /g' -

In [ ]:
## Done
## 161123 SRA files processed

In [ ]:
# On EC2
# Download all summaries
aws s3 sync s3://serratus-public/out/200830_pmeta5/psummary/ ./

S3DIR='s3://serratus-public/notebook/200830_ab'

function getViro {
VNAME=$1
# Grab viral family from all summary files
grep -r $VNAME \
  | grep 'seqcvg' - \
  | sort  -  \
  > ../$VNAME.pmeta5.psum
  
  aws s3 cp ../$VNAME.pmeta5.psum $S3DIR/psum/
  
}

getViro Corona
getViro Epsy
getViro Medion
getViro Roni
getViro Delta
getViro Hepe
getViro Dicistro
getViro Quenya


In [ ]:
# Manual reduce down to 'interesting cases'
cp ../$VNAME.pmeta5.psum ../$VNAME.assembly.psum
# vim $VNAME.assembly.psum

# Reduce assembly list to SRA entries only
sed 's/.*sra=//g' $VNAME.assembly.psum \
  | sed 's/;.*//g' - \
  | sort -k1 - \
  | uniq \
  > $VNAME.assembly.sra